In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
!pip install lightning
import lightning as L

class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()
        div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0), :]

class Attention(nn.Module): 
    def __init__(self, d_model=2):
        super().__init__()
        self.d_model=d_model
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model)
        self.row_dim = 0
        self.col_dim = 1
    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)
        sims = torch.matmul(q, k.T)
        scaled_sims = sims / torch.sqrt(torch.tensor(self.d_model))
        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, v)
        return attention_scores

class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=6, seed=42):
        super().__init__()
        L.seed_everything(seed=seed)
        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)     
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
        self.self_attention = Attention(d_model=d_model)
        self.fc_change_direction = nn.Linear(in_features=d_model, out_features=d_model)
        self.fc_output = nn.Linear(in_features=d_model, out_features=num_tokens)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)        
        inputs_position_encoded = self.pe(word_embeddings)
        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0)), device=self.device))
        mask = mask == 0
        self_attention_values = self.self_attention(inputs_position_encoded, 
                                                    inputs_position_encoded, 
                                                    inputs_position_encoded, 
                                                    mask=mask)
        
        residual_connection_values = inputs_position_encoded + self_attention_values
        change_directions = self.fc_change_direction(residual_connection_values)
        residual_connection_values = residual_connection_values + change_directions

        fc_layer_output = self.fc_output(residual_connection_values)
        return fc_layer_output
    
    def configure_optimizers(self): 
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx): 
        input_tokens, labels = batch
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])
        return loss

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

!pip install lightning
import lightning as L

# inspired by https://github.com/StatQuest/decoder_transformer_from_scratch

token_to_id = {'what' : 0,
               'is' : 1,
               'DL' : 2,
               'awesome': 3,
               '<EOS>' : 4,
              }

id_to_token = {v: k for k, v in token_to_id.items()}

sentence1 = [token_to_id["what"],
            token_to_id["is"], 
            token_to_id["DL"], 
            token_to_id["<EOS>"],
            token_to_id["awesome"]]

sentence2 = [token_to_id["DL"],
            token_to_id["is"], 
            token_to_id["what"], 
            token_to_id["<EOS>"], 
            token_to_id["awesome"]]

inputs = torch.tensor([sentence1, sentence2])

label1 = [token_to_id["is"], 
        token_to_id["DL"], 
        token_to_id["<EOS>"], 
        token_to_id["awesome"], 
        token_to_id["<EOS>"]]

label2 = [token_to_id["is"], 
        token_to_id["what"], 
        token_to_id["<EOS>"], 
        token_to_id["awesome"], 
        token_to_id["<EOS>"]]

labels = torch.tensor([label1,  
                       label2
                       ])

dataset = TensorDataset(inputs, labels) 
dataloader = DataLoader(dataset)

model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6, seed=43)

model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["DL"], 
                            token_to_id["<EOS>"]])

def run_prompt(model_input: torch.tensor, model: L.LightningModule):
    print(">>> ", " ".join([id_to_token[i.item()] for i in model_input]))

    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = predicted_id
    input_length = model_input.size(dim=0)

    max_length = 6
    for i in range(input_length, max_length):
        if (predicted_id == token_to_id["<EOS>"]):
            break

        model_input = torch.cat((model_input, predicted_id))
        
        predictions = model(model_input) 
        predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
        predicted_ids = torch.cat((predicted_ids, predicted_id))

    print("Predicted Tokens:\n") 
    print(predicted_ids)
    for id in predicted_ids: 
        print("\t", id_to_token[id.item()])

def train_model(model: L.LightningModule, dataloader: DataLoader):
    trainer = L.Trainer(max_epochs=30)
    trainer.fit(model, train_dataloaders=dataloader)

print("*** BEFORE TRAINING ***")
run_prompt(model_input, model)
print("*** RUNNING TRAINING ***")
train_model(model, dataloader)
print("*** AFTER TRAINING ***")
run_prompt(model_input, model)


Seed set to 43
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Schiendorfer\Anaconda3\envs\llm\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | we                  | Embedding        | 1

*** BEFORE TRAINING ***
>>>  what is DL <EOS>
Predicted Tokens:

tensor([2, 1, 4])
	 DL
	 is
	 <EOS>
*** RUNNING TRAINING ***
Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 135.84it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 101.40it/s, v_num=2]
*** AFTER TRAINING ***
>>>  what is DL <EOS>
Predicted Tokens:

tensor([3, 4])
	 awesome
	 <EOS>
